Database ve tabloların oluşturulup kayıtların eklenmesi

In [1]:
import psycopg2

# Global olarak erişilecek nesneler
conn = None
cur = None

def baslat():
    global conn, cur
    conn = psycopg2.connect(
        host="localhost",
        dbname="homework",
        user="postgres",
        password="1234",
        port="5432"
    )
    cur = conn.cursor()
    print("Veritabanı bağlantısı açıldı.")

def bitir():
    global conn, cur
    if cur:
        cur.close()
    if conn:
        conn.commit()
        conn.close()
        print("Veritabanı bağlantısı kapatıldı.")


In [2]:
baslat()

cur.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        emp_id INT PRIMARY KEY,
        first_name VARCHAR(50),
        last_name VARCHAR(50),
        salary INT,
        job_title VARCHAR(55),
        gender VARCHAR(6) CHECK (gender IN ('Female', 'Male')),
        hire_date DATE
    );

    CREATE TABLE IF NOT EXISTS departments (
        dept_id INT,
        dept_name VARCHAR(50),
        emp_id INT REFERENCES employees(emp_id)
    );
""")

print("Tablolar oluşturuldu (varsa zaten dokunulmadı).")

bitir()


Veritabanı bağlantısı açıldı.
Tablolar oluşturuldu (varsa zaten dokunulmadı).
Veritabanı bağlantısı kapatıldı.


In [3]:
baslat()
cur.execute("""
    INSERT INTO employees (emp_id, first_name, last_name, salary, job_title, gender, hire_date) VALUES
    (17679, 'Robert', 'Gilmore', 110000, 'Operations Director', 'Male', '2018-09-04'),
    (26650, 'Elvis', 'Ritter', 86000, 'Sales Manager', 'Male', '2017-11-24'),
    (30840, 'David', 'Barrow', 85000, 'Data Scientist', 'Male', '2019-12-02'),
    (49714, 'Hugo', 'Forester', 55000, 'IT Support Specialist', 'Male', '2019-11-22'),
    (51821, 'Linda', 'Foster', 95000, 'Data Scientist', 'Female', '2019-04-29'),
    (67323, 'Lisa', 'Wiener', 75000, 'Business Analyst', 'Female', '2018-08-09'),
    (70950, 'Rodney', 'Weaver', 87000, 'Project Manager', 'Male', '2018-12-20'),
    (71329, 'Gayle', 'Meyer', 77000, 'HR Manager', 'Female', '2019-06-28'),
    (76589, 'Jason', 'Christian', 99000, 'Project Manager', 'Male', '2019-01-21'),
    (97927, 'Bille', 'Lanning', 67000, 'Web Developer', 'Female', '2018-06-25');
""")

cur.execute("""
    INSERT INTO departments (dept_id, dept_name, emp_id) VALUES
    (13, 'Operations', 17679),
    (14, 'Marketing', 26650),
    (13, 'Operations', 30840),
    (12, 'Technology', 49714),
    (13, 'Operations', 51821),
    (14, 'Marketing', 67323),
    (11, 'Administrative', 70950),
    (13, 'Operations', 76589),
    (12, 'Technology', 97927);
""")

bitir()

Veritabanı bağlantısı açıldı.
Veritabanı bağlantısı kapatıldı.


1 - Find the employees who get paid more than Rodney Weaver.

In [4]:
baslat()

cur.execute("""SELECT * FROM employees 
            WHERE salary > (SELECT salary FROM employees WHERE first_name = 'Rodney' AND last_name = 'Weaver');
""")

for row in cur.fetchall():
    print(row)

bitir()

Veritabanı bağlantısı açıldı.
(17679, 'Robert', 'Gilmore', 110000, 'Operations Director', 'Male', datetime.date(2018, 9, 4))
(51821, 'Linda', 'Foster', 95000, 'Data Scientist', 'Female', datetime.date(2019, 4, 29))
(76589, 'Jason', 'Christian', 99000, 'Project Manager', 'Male', datetime.date(2019, 1, 21))
Veritabanı bağlantısı kapatıldı.


2 - Find the average, min and max salaries

In [5]:
baslat()

cur.execute("""
    SELECT 
        ROUND(AVG(salary), 2) AS average_salary,
        MAX(salary) AS max_salary,
        MIN(salary) AS min_salary
    FROM employees;
""")
result = cur.fetchall()
print(f"ortalama maaş: {result[0][0]}")
print(f"maximum maaş: {result[0][1]}")
print(f"minimum maaş: {result[0][2]}")

bitir()

Veritabanı bağlantısı açıldı.
ortalama maaş: 83600.00
maximum maaş: 110000
minimum maaş: 55000
Veritabanı bağlantısı kapatıldı.


3 - Find the employees whose salary is more than 8700. Our query should return first name, last name, and salary info of the employees.

In [6]:
baslat()

cur.execute("""SELECT first_name, last_name, salary FROM employees
WHERE salary > 87000""")
result = cur.fetchall()
for a,b,c in result:
    print(a,b,"-",c)

bitir()

Veritabanı bağlantısı açıldı.
Robert Gilmore - 110000
Linda Foster - 95000
Jason Christian - 99000
Veritabanı bağlantısı kapatıldı.


4 - Find the employees (first name, last name from employees table) who work under the Operations department (departments table). Our query should return first name and last name info.

In [7]:
baslat()

cur.execute("""SELECT first_name, last_name
FROM departments JOIN employees
ON departments.emp_id = employees.emp_id;""")
for a,b in cur.fetchall():
    print(a,b)

Veritabanı bağlantısı açıldı.
Robert Gilmore
Elvis Ritter
David Barrow
Hugo Forester
Linda Foster
Lisa Wiener
Rodney Weaver
Jason Christian
Bille Lanning


5 - Find the employees (first name, last name from employees table) who work under the Technology department (departments table). Our query should return first name and last name info.

In [8]:
baslat()

cur.execute("""SELECT first_name, last_name
FROM departments JOIN employees
ON departments.emp_id = employees.emp_id
WHERE departments.dept_name = 'Technology';""")

for a,b in cur.fetchall():
    print(a,b)

Veritabanı bağlantısı açıldı.
Hugo Forester
Bille Lanning


6 - Find the average salary of female employees.

In [9]:
baslat()

cur.execute("""SELECT ROUND(AVG(salary),2)
FROM employees
WHERE gender = 'Female';""")

ortalama = cur.fetchone()
print(ortalama[0])

bitir()

Veritabanı bağlantısı açıldı.
78500.00
Veritabanı bağlantısı kapatıldı.


7 - Find the average salaries of each department.

In [10]:
baslat()

cur.execute("""SELECT departments.dept_name AS "Departman Adı", ROUND(AVG(salary),2) AS "Ortalama_maaş"
FROM employees JOIN departments
ON employees.emp_id = departments.emp_id 
GROUP BY departments.dept_name;""")

rows = cur.fetchall()

print("Departman Adı - Ortalama Maaş")
print("------------------------------")

for row in rows:
    print(f"{row[0]} - {row[1]}")

bitir()

Veritabanı bağlantısı açıldı.
Departman Adı - Ortalama Maaş
------------------------------
Technology - 61000.00
Marketing - 80500.00
Operations - 97250.00
Administrative - 87000.00
Veritabanı bağlantısı kapatıldı.


8 - Find the oldest and newest employees.

In [14]:
baslat()

cur.execute("""SELECT * FROM employees
ORDER BY hire_date ASC
LIMIT 1;""")

print(f"En uzun Çalışan:\n{cur.fetchone()}")

cur.execute("""SELECT * FROM employees
ORDER BY hire_date DESC
LIMIT 1;""")

print(f"En kısa Çalışan:\n{cur.fetchone()}")

bitir()

Veritabanı bağlantısı açıldı.
En uzun Çalışan:
(26650, 'Elvis', 'Ritter', 86000, 'Sales Manager', 'Male', datetime.date(2017, 11, 24))
En kısa Çalışan:
(30840, 'David', 'Barrow', 85000, 'Data Scientist', 'Male', datetime.date(2019, 12, 2))
Veritabanı bağlantısı kapatıldı.


9 - Find the hiring date and department of the highest paid employee

In [16]:
baslat()

cur.execute("""SELECT hire_date, dept_name
FROM employees JOIN departments
ON employees.emp_id = departments.emp_id
ORDER BY salary
LIMIT 1;""")

result = cur.fetchall()

print(f"En yüksek maaş alanın işe giriş tarihi: {result[0][0]}")
print(f"En yüksek maaş alanın departman adı: {result[0][1]}")

bitir()

Veritabanı bağlantısı açıldı.
En yüksek maaş alanın işe giriş tarihi: 2019-11-22
En yüksek maaş alanın departman adı: Technology
Veritabanı bağlantısı kapatıldı.


10 - Find the hiring date and department of the lowest paid employee

In [17]:
baslat()

cur.execute("""SELECT hire_date, dept_name
FROM employees JOIN departments
ON employees.emp_id = departments.emp_id
ORDER BY salary DESC
LIMIT 1;""")

result = cur.fetchall()

print(f"En düşük maaş alanın işe giriş tarihi: {result[0][0]}")
print(f"En düşük maaş alanın departman adı: {result[0][1]}")

bitir()

Veritabanı bağlantısı açıldı.
En düşük maaş alanın işe giriş tarihi: 2018-09-04
En düşük maaş alanın departman adı: Operations
Veritabanı bağlantısı kapatıldı.
